# autocomplete data test

In [ ]:
# data test for autocomplete
import json

with open('words_autocomplete1.json', 'r') as fp:
    w_autocomplete = json.load(fp)

In [ ]:
# randomnly select 200 words from 465142 1 gram and 2 gram in datasets and NIFS Ontology
# - select word with length between 3 and 15
# - word do not contain number

import random
import re
import json

words = list(w_autocomplete.keys())
testwords = []
for i in range(200):
    while True:
        pos = random.randint(0, len(w_autocomplete)-1)
        word = words[pos]
        if '/' not in word and len(word)>2 and len(word) < 15 and not bool(re.search(r'\d', word)) and len(word.split())==1:
            testwords += [word]
            break

# save data test to file
with open('test_autocomplete_pure.json', 'w') as fp:
    json.dump(testwords, fp)
    
# prepare data for 1 typo
test_1typo = []
for word in testwords:
    pos = random.randint(1, len(word)-1) 
    test_1typo += [word[:pos]+'*'+word[pos+1:]]

# save data test typo to file
with open('test_autocomplete_typo.json', 'w') as fp:
    json.dump(test_1typo, fp)

# query data test

We generated test collections by randomly select keywords and authors related to the SPARC data sets, including keywords and authors. Hence,  test collection is a pair of query and a list of the corresponding data sets. Then, we differentiated the test collections based on the number of terms in the query and mimicking typos by performing insertion, deletion, replacement, and spaces removal. In total,there are 31 test collections consisting 50 pairs of query and a list of datasets.

Here are the type of the test collection number of terms in the query we used:
- keyword
    - 1 term query
        - no typo
        - 1 deletion
        - 1 insertion
        - 1 replacement
    - 2 terms query
        - no typo
        - 1 deletion
        - 1 insertion
        - 1 replacement
        - no space
        - no space with 1 typo
        - no space with 2 typos
        - no space with 3 typos
        - 3 typos
    - 3 terms query
        - no typo
        - 1 deletion
        - 1 insertion
        - 1 replacement
        - no space
        - no space with 1 typo
        - no space with 2 typos
        - no space with 3 typos
        - 3 typos
- author
    - 1 term query
        - no typo
        - 1 deletion
        - 1 insertion
        - 1 replacement
    - 2 terms query
        - no typo
        - 1 deletion
        - 1 insertion
        - 1 replacement
        - no space

In [348]:
# a function to retrieve information from json using path
def __find(element, JSON):
    """
    To find the content in elasticsearch's hits based on path in element.
    Arguments:
        - element: a path to the content, e.g. _source.item.keywords.keyword
        - JSON: a dictionary as a result of elasticsearch query
    """
    try:
        paths = element.split(".")
        data = JSON
        for count, p in enumerate(paths):
            if isinstance(data[p], dict):
                data = data[p]
            elif isinstance(data[p], list):
                data = [__find(element[element.find(p)+len(p)+1:], lst) for lst in data[p]]
                break
            elif len(paths)-1 == count:
                return data[p]
        if [] in data: data.remove([])
        return data
    except:
        return []
    
# open a file containing the metadata of the datasets
import json
with open('sparc_datasets.json', 'r') as fp:
    sparc_meta = json.load(fp)

In [349]:
# organise metadata
term2dataset = {} # term to list of dataset (the location in hit list is used to determine id, not the dataset identifier)
keyword2dataset = {} # keyword to list of dataset
keywords = {} # keyword based on length {1:{...}, 2:{...}, ...}

author2dataset = {} # author to list of dataset
sname2dataset = {} # name to list of dataset

for hit in enumerate(sparc_meta['hits']['hits']):
    try:
        idx = hit['_source']['pennsieve']['identifier']
    except:
        continue
    
    for keyword in __find('_source.item.keywords.keyword', hit):
        keyword = keyword.lower()
        length = len(keyword.split())
        if length not in keywords: keywords[length] = []
        keywords[length] += [keyword]
        if keyword not in keyword2dataset: keyword2dataset[keyword] = []
        keyword2dataset[keyword] += [idx]
        for term in keyword.split():
            if term not in term2dataset: term2dataset[term] = []
            term2dataset[term] += [idx]
    for length in keywords:
        keywords[length] = list(set(keywords[length]))
            
    firsts = __find('_source.contributors.first.name', hit)
    lasts = __find('_source.contributors.last.name', hit)
    for name in [first+' '+last for first, last in (zip(firsts, lasts))]:
        name = name.lower()
        if name not in author2dataset: author2dataset[name] = []
        author2dataset[name] += [idx]
    for first in firsts:
        first = first.lower()
        if first not in sname2dataset: sname2dataset[first] = []
        sname2dataset[first] += [idx]
    for last in lasts:
        last = last.lower()
        if last not in sname2dataset: sname2dataset[last] = []
        sname2dataset[last] += [idx]

In [350]:
# a function to generate datatest
# parameters: - numOfTerm (int) : the number of term in a query
#             - numOfData (int) : the number of generated test queries

import random
import copy

def generateDataTest(numOfTerm, numOfData):
    dataTest = {}
    if numOfData < len(keywords[numOfTerm]):
        kws = copy.deepcopy(keywords[numOfTerm])
        while len(dataTest) < numOfData:
            keyword = random.choice(kws)
            dataTest[keyword] = list(set(keyword2dataset[keyword]))
            kws.remove(keyword)
    else:
        for keyword in keywords[numOfTerm]:
            dataTest[keyword] = list(set(keyword2dataset[keyword]))
        
        kws = list(term2dataset.keys())
        while len(dataTest) < numOfData:
            if numOfTerm in [1, 2]:
                keyword = ''
                idxs = []
                for i in range(numOfTerm):
                    term = ''
                    while len(term)<=3:
                        term = random.choice(kws)
                    keyword += term + ' '
                    idxs += term2dataset[term]
                keyword = keyword.strip()
                if keyword not in dataTest:
                    dataTest[keyword] = list(set(idxs))
            elif numOfTerm == 3:
                keyword = ''
                term1 = random.choice(list(keywords[2]))
                term2 = random.choice(kws)
                keyword = term1 + ' ' + term2
                if keyword not in dataTest:
                    dataTest[keyword] = list(set(keyword2dataset[term1]))
    return dataTest

In [352]:
# create 1 terms dataset
query_1_terms = generateDataTest(1, 50)

In [353]:
# create 2 terms dataset
query_2_terms = generateDataTest(2, 50)

In [354]:
# create 3 terms dataset
query_3_terms = generateDataTest(3, 50)

In [355]:
import copy
import string
# dataset 1 term 1 deletion
def get_1_deletion(oldData):
    newData = copy.deepcopy(oldData)
    for query in list(newData.keys()):
        pos = random.randint(0, len(query)-1)
        newQuery = query[:pos] + query[pos+1:]
        newData[newQuery] = newData[query]
        del newData[query]
    return newData
    
# dataset 1 term 1 insertion
def get_1_insertion(oldData):
    newData= copy.deepcopy(oldData)
    for query in list(newData.keys()):
        pos = random.randint(0, len(query)-1)
        char = random.choice(string.ascii_lowercase)
        newQuery = query[:pos] + char + query[pos:]
        newData[newQuery] = newData[query]
        del newData[query]
    return newData
    
# dataset 1 term 1 replacement
def get_1_replacement(oldData):
    newData= copy.deepcopy(oldData)
    for query in list(newData.keys()):
        pos = random.randint(0, len(query)-1)
        char = random.choice(string.ascii_lowercase)
        newQuery = query[:pos] + char + query[pos+1:]
        newData[newQuery] = newData[query]
        del newData[query]
    return newData

# remove space
def removeSpace(oldData):
    newData = copy.deepcopy(oldData)
    for query in list(newData.keys()):
        newQuery = query.replace(' ','')
        newData[newQuery] = newData[query]
        del newData[query]
    return newData

# 1 random distance
def get_1_distance(oldData):
    newData= copy.deepcopy(oldData)
    for query in list(newData.keys()):
        pos = random.randint(0, len(query)-1)
        char = random.choice(string.ascii_lowercase)
        distType = random.randint(0,2)
        if distType == 0:
            newQuery = query[:pos] + query[pos+1:]
        elif distType == 1:
            newQuery = query[:pos] + char + query[pos:]
        else:
            newQuery = query[:pos] + char + query[pos+1:]
        newData[newQuery] = newData[query]
        del newData[query]
    return newData

In [357]:
datatest = {
    1:{
        '0 typo': query_1_terms,
        '1 del': get_1_deletion(query_1_terms),
        '1 insert': get_1_insertion(query_1_terms),
        '1 replace': get_1_replacement(query_1_terms),
    },
    2:{
        '0 typo': query_2_terms,
        '1 del': get_1_deletion(query_2_terms),
        '1 insert': get_1_insertion(query_2_terms),
        '1 replace': get_1_replacement(query_2_terms),
        'no space': removeSpace(query_2_terms),
        'no space 1 typo': get_1_distance(removeSpace(query_2_terms)),
        'no space 2 typo': get_1_distance(get_1_distance(removeSpace(query_2_terms))),
        'no space 3 typo': get_1_deletion(get_1_insertion(get_1_replacement(removeSpace(query_2_terms)))),
        '3 typo': get_1_deletion(get_1_insertion(get_1_replacement(query_2_terms))),
    },
    3:{
        '0 typo': query_3_terms,
        '1 del': get_1_deletion(query_3_terms),
        '1 insert': get_1_insertion(query_3_terms),
        '1 replace': get_1_replacement(query_3_terms),
        'no space': removeSpace(query_3_terms),
        'no space 1 typo': get_1_distance(removeSpace(query_3_terms)),
        'no space 2 typo': get_1_distance(get_1_distance(removeSpace(query_3_terms))),
        'no space 3 typo': get_1_deletion(get_1_insertion(get_1_replacement(removeSpace(query_3_terms)))),
        '3 typo': get_1_deletion(get_1_insertion(get_1_replacement(query_3_terms))),
    }
}

In [253]:
# save datatests
import json

with open('query_datatest.json', 'w') as fp:
    json.dump(datatest, fp)

In [173]:
# create author datatest
author_1_terms = {}
while len(author_1_terms) < numOfData:
    keyword = random.choice(list(sname2dataset.keys()))
    if keyword not in author_1_terms:
        author_1_terms[keyword] = list(set(sname2dataset[keyword]))

author_2_terms = {}
while len(author_2_terms) < numOfData:
    keyword = random.choice(list(author2dataset.keys()))
    if keyword not in author_2_terms:
        author_2_terms[keyword] = list(set(author2dataset[keyword]))



In [175]:
datatest_author = {
    1:{
        '0 typo': author_1_terms,
        '1 del': get_1_deletion(author_1_terms),
        '1 insert': get_1_insertion(author_1_terms),
        '1 replace': get_1_replacement(author_1_terms)
    },
    2:{
        '0 typo': author_2_terms,
        '1 del': get_1_deletion(author_2_terms),
        '1 insert': get_1_insertion(author_2_terms),
        '1 replace': get_1_replacement(author_2_terms),
        'no space': removeSpace(author_2_terms)
    }
}

In [178]:
# save author datatests
import json

with open('author_datatest.json', 'w') as fp:
    json.dump(datatest_author, fp)